## Use Jupyter notebook as collaboration tool

### Learning Python by web scraping

In [1]:
####################################################
# Goal : Scraping the data on Zika virus           #
# Author : Jean-Claude Razafindrakoto              #
# Date : 2016-09-06                                #
# Python : 3.5.2 (Anaconda 4.1.11)                 #
####################################################

# Libraries import
from lxml import html
import requests
import pandas as pd
# FOR FUNCTIONAL PROGRAMMING
#import cytoolz  # pipe
#import cytoolz.curried  # map